The purpose of this notebook is to incoporate some of Fred Bahr's inital station operation status code into a more modular framework to eventually feed in to a dashboard/UI. <br>
Marine Lebrec, June 2024

In [33]:
import numpy as np
import xarray as xr
import pandas as pd
import datetime as dt
from datetime import timedelta
import requests
import json
import sys
import smtplib
from email.message import EmailMessage
import csv

datetime.datetime(2024, 6, 18, 22, 10, 6, 813376, tzinfo=datetime.timezone.utc)

**Checklist**
- Generate list of station names (long nam & short name for erddap url) - DONE
- 

In [80]:
# Get list of all stations that we are interested in.
def get_all_stations():
    station_file = 'station_names.json'
    f = open(station_file)
    data = json.load(f)
    stationnames = []
    for i in data:
        stationnames.append(i['stationName'])
    return stationnames

def get_erddapid(station_file, station_name):
    f = open(station_file)
    data = json.load(f)

    for station in data:
            if station['stationName'] == station_name:
                return station['datasetID']
            
def get_timedelta(erddapID):
    now = dt.datetime.now(tz=dt.timezone.utc)
    df = pd.read_csv(f'https://erddap.sensors.axds.co/erddap/tabledap/{erddapID}.csv?time')
    last_time = dt.datetime.strptime(df['time'].iloc[-1],'%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=dt.timezone.utc)
    time_delta = now - last_time

    # parse for more meaningful output
    days = time_delta.days
    hours, remainder = divmod(time_delta.seconds, 3600)
    minutes, _ = divmod(remainder, 60)

    # Construct the timedelta string without seconds
    if days > 0:
        timedelta_str = f"{days} days, {hours} hours, {minutes} minutes"
    elif hours > 0:
        timedelta_str = f"{hours} hours, {minutes} minutes"
    elif minutes > 0:
        timedelta_str = f"{minutes} minutes"
    else:
        timedelta_str = "< 1 minute"
    
    print(timedelta_str)
    return timedelta_str

def write_data_to_csv(station, timedelta_str, outputfile):
     with open(outputfile, 'a', newline='') as csvfile:
          fieldnames = ['stationName', 'timeDelta']
          writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
          if csvfile.tell() == 0:
              writer.writeheader()
          writer.writerow({'stationName': station, 'timeDelta': timedelta_str})
          print(f'{station} {timedelta_str} written to csv')
     
get_erddapid('station_names.json', 'Santa Cruz municipal wharf')

'edu_ucsc_scwharf1'

In [81]:
outputfile = 'stations_timedelta.csv'
for station in get_all_stations():
    erddapid = get_erddapid(station_file = 'station_names.json', station_name = station)
    timedelta_str = get_timedelta(erddapID=erddapid)
    write_data_to_csv(station = station, timedelta_str = timedelta_str, outputfile = outputfile)

    #print(erddapid, time_delta)


2 hours, 17 minutes
Moss Landing Marine Laboratories Seawater Intake Monitoring Station (MLSC1) 2 hours, 17 minutes written to csv
847 days, 18 hours, 10 minutes
Romberg Tiburon Center Pier 847 days, 18 hours, 10 minutes written to csv
1323 days, 5 hours, 43 minutes
Santa Cruz municipal wharf 1323 days, 5 hours, 43 minutes written to csv
23 minutes
Trinidad Head, California 23 minutes written to csv
229 days, 6 hours, 5 minutes
Bodega Marine Laboratory seawater intake 229 days, 6 hours, 5 minutes written to csv
11 days, 0 hours, 23 minutes
Carquinez 11 days, 0 hours, 23 minutes written to csv
229 days, 9 hours, 36 minutes
Fort Point Pier 229 days, 9 hours, 36 minutes written to csv
24 minutes
Humboldt Bay Pier 24 minutes written to csv
1663 days, 7 hours, 39 minutes
Monterey Bay Commercial Wharf 1663 days, 7 hours, 39 minutes written to csv
24 minutes
Morro Bay - BM1 T-Pier 24 minutes written to csv
24 minutes
Morro Bay - BS1 24 minutes written to csv
42 minutes
San Luis Bay - Cal Poly

In [72]:
erddapid = get_erddapid(station_file = 'station_names.json', station_name = station)
time_delta = get_timedelta(erddapID=erddapid)
type(time_delta)

datetime.timedelta